In [ ]:
from __future__ import print_function
import os.path
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, '../../JKBio/')
import CCLF_processing
from IPython.display import Image, display, HTML

# Import requirements for making CNV plots
from matplotlib import pyplot as plt
import cnvlib

# Import requirements for interactive features
import qgrid
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, interactive
import re # used for regex

# Extra options
qgrid.set_defaults(remote_js=True, precision=3, export_mode=True)
qgrid.set_grid_option('maxVisibleRows', 10)

In [ ]:
# Using Phil's code for card view so I can have a dropdown in a static HTML. This is in the next three chunks.


In [ ]:
%%javascript

var script = document.createElement('script');
script.type = 'text/javascript';
script.src = '//code.jquery.com/jquery-3.3.1.min.js';
document.head.appendChild(script);

function updateVisibleCard(id_prefix) {
    $("."+id_prefix).hide()
    $("#"+$("#"+id_prefix+"-select").children("option:selected").val()).show()
}

console.log(window)
window.updateVisibleCard = updateVisibleCard

In [ ]:
import IPython
from IPython import get_ipython
def _get_html(obj):
    """Get the HTML representation of an object"""
    # TODO: use displaypub to make this more general
    ip = get_ipython()
    png_rep = ip.display_formatter.formatters['image/png'](obj)

    if png_rep is not None:
        #if isinstance(obj, plt.Figure):
        #    plt.close(obj)  # keep from displaying twice
        return ('<img src="data:image/png;'
                'base64,{0}">'.format(png_rep))
    else:
        return "<p> {0} </p>".format(str(obj))

    rep = ip.display_formatter.formatters['text/html'](obj)

    if rep is not None:
        return rep
    elif hasattr(obj, '_repr_html_'):
        return obj._repr_html_()

In [ ]:
import random
import string

def random_id(n):
    return ''.join([random.choice(string.ascii_uppercase+string.ascii_lowercase) for _ in range(n)])

def _select_html(id_prefix, labels):
    select_tmpl = "<select id=\"{id_prefix}-select\" onchange=\"updateVisibleCard('{id_prefix}')\">{options}</select>"
    option_tmpl = "<option value=\"{id_prefix}-{i}\" {selected}>{label}</option>"
    options = "".join([option_tmpl.format(selected={True: "selected", False: ""}[i == 0],
                                          i=i, label=label, id_prefix=id_prefix) for i, label in enumerate(labels)])
    return select_tmpl.format(**locals())

def _cards_html(id_prefix, images):
    div_tmpl = "<div class=\"{id_prefix}\" id=\"{id_prefix}-{i}\" {style}>{inner_html}</div>"
    img_divs = [div_tmpl.format(inner_html=_get_html(image), 
                                style={False: "style=\"display: none\"", True: ""}[i == 0],
                                id_prefix=id_prefix, 
                                i=i) for i, image in enumerate(images)]
    return "".join(img_divs)

def _select_card_html(labels, images):
    assert len(labels) == len(images)
    id_prefix = random_id(8)
    select_html = _select_html(id_prefix, labels)
    cards_html = _cards_html(id_prefix, images)
    return select_html + cards_html

def display_card_view(labels, images):
    return IPython.display.HTML(_select_card_html(labels, images))
    
#IPython.display.HTML("""
#<select id="zzz-select">
#<option value="zzz-1" selected>google</option>
#<option value="zzz-2">bird</option>
#</select>
#<div style='background-color:red; width: 100px'>aaa<div class='zzz' id='zzz-1'>"""
#                     +_get_html(i1)+"</div><div class='zzz' id='zzz-2' style='display: none'>"+_get_html(i2)+"</div>")

In [ ]:
cwd = os.getcwd()
print(cwd)

In [ ]:
participant = "PEDS204"

# path would be the participant-specific path
# path = "gs://cclf_results/targeted/neekesh_201912/" 
path = "gs://cclf_results/targeted/neekesh_201912/Wilms_Tumor/PEDS204/"

disease = str(os.path.basename(os.path.dirname(os.path.dirname(path)))).replace("_", " ")

***
***

# Report for participant: {{participant}} ({{disease}})

In [ ]:
def read_in_tables(filepaths):
    """ takes list of filepaths to TSVs and returns dataframes read in by Pandas"""
    return [pd.read_csv(f, sep='\t', index_col = None) for f in filepaths]

In [ ]:
%%capture
# A list of file paths for the selected participant
filepaths = ! gsutil ls -r {path}**

# Get all the table filepaths in the bucket
table_filepaths = ! gsutil ls -r {path}*.txt # check: will this search recursively for all .txt files?
to_add = ! gsutil ls -r {path}**.tsv
table_filepaths += to_add

# Get all the png filepaths in the bucket
img_filepaths = ! gsutil ls -r {path}**.png

# Copy all the pngs in the bucket to a tmp folder
# TODO: need to delete the files afterwards
tempdir='../temp/cclfreport/images/'
! gsutil cp -r {path}**.png {tempdir} # copy images from google bucket to local temp folder
# local_img_filepaths = ! ls {tempdir}*.png
local_img_filepaths = [tempdir + os.path.basename(i) for i in img_filepaths]

# TODO: currently not exactly the external ID - it's the name of the image itself.
# Dictionary mapping external IDs to their respective horizontal CN plot GS links
ext_id_to_GS_img = {os.path.basename(link)[:-4]:link for link in img_filepaths}

# Directory mapping external IDs to the local filepaths for the horizontal CN plots
ext_id_to_local_img = {os.path.basename(link)[:-4]:link for link in local_img_filepaths}

In [ ]:
def make_interactive_table(filepath, cols_to_include = None):
    """Takes single pd dataframe as input"""
    if type(filepath) != pd.core.frame.DataFrame:
        raise Exception("The function expected a pandas dataframe as input, but got: ", str(type(filepath)))
    data = filepath
    
    # Subset the data to include the specified columns, if any passed in
    if cols_to_include is not None:
        # The index is the first column listed
        index_name = cols_to_include[0]
        data = data[cols_to_include]
        data.set_index(index_name, inplace=True, drop=True)
        if 'keep' in cols_to_include:
            data = data.loc[data['keep'] == True]
            
    # Create and display interactive table
    qgrid_widget = qgrid.show_grid(data, show_toolbar=False, grid_options = {'forceFitColumns': False,
    'defaultColumnWidth': 150}, precision=3)
    display(qgrid_widget)
    print("\n")

***
***

# Sample information and identifiers
This section details the external IDs for all the samples we discovered when searching the existing targeted probe data and WES data.

In [ ]:
# Sample information and identifiers
all_external_ids = ! gsutil ls -r {path}**all_external_ids.tsv
all_failed_external_ids = ! gsutil ls -r {path}**all_failed_external_ids.tsv

# Read in the tables
all_external_ids_df = read_in_tables(all_external_ids)
all_failed_external_ids_df = read_in_tables(all_failed_external_ids)

## Table: all external IDs & associated metadata
The below table is sortable and filterable. You can double-click on the cells in the table if you want to copy the contents, like if you wanted to copy the link to the file in the Google storage console.

In [ ]:
# Make a table with metadata for each external ID

summary_df = all_external_ids_df[0]
summary_df['participant'] = str(os.path.basename(os.path.dirname(all_external_ids[0])))
summary_df['disease'] = str(os.path.basename(os.path.dirname(os.path.dirname(all_external_ids[0]))))
summary_df['filepath'] = str(os.path.dirname(all_external_ids[0]))
summary_df['link'] = str(re.sub('gs://', 'https://console.cloud.google.com/storage/browser/', os.path.dirname(all_external_ids[0])))

summary_df.set_index('participant', drop=True, inplace=True)

# Print some summary information
print("We found a total of", summary_df.shape[0],"external IDs that passed the depth of coverage QC.")
print("Do note that some of these samples may not have been processed yet, and thus may not have CN plots or mutation information.")
# TODO: add metadata showing whether we have processed CN, SNV, etc. Ask what other metadata would be useful.

# Display an interactive table
qgrid_widget = qgrid.show_grid(summary_df, show_toolbar=False, grid_options = {'forceFitColumns': False})
display(qgrid_widget)

## Samples that failed the depth of coverage QC
This summary details all the the external IDs of each sample that failed the depth of coverage QC in the targeted probe pipeline. The depth of coverage QC in the targeted probe pipeline requires that the average gene-level or interval-level coverage is >=50x. 

The summary also lists the participants for which no samples failed the depth of coverage QC.

In [ ]:
# Print the external IDs that passed the QC
failed_qc = False
for table,filepath in zip(all_failed_external_ids_df, all_failed_external_ids):
    tmp_df = table
    participant_name = str(os.path.basename(os.path.dirname(filepath)))
    if tmp_df.shape[0] ==1:
        print("There was", str(tmp_df.shape[0]), "failed sample for participant", participant_name,":")
        display(sorted(tmp_df.iloc[:,0].tolist()))
    elif tmp_df.shape[0] >1:
        print("There were", str(tmp_df.shape[0]), "failed samples for participant", participant_name)
    else:
        failed_qc = True

# If any samples failed the QC, print them:
if failed_qc:
    print("There were no failed samples for participant", participant_name)

***
***

# Copy number data

In [ ]:
# Get the CN tables from the Google storage bucket and create list of paths
tsca_cn = ! gsutil ls -r {path}copy_number.tsv
wes_cn = ! gsutil ls -r {path}**wes_copy_number.tsv
all_cn_tables = wes_cn + tsca_cn

# Create dictionary with filepaths as keys and pandas DF as the values
tsca_cn_dict = {f:pd.read_csv(f, sep="\t", index_col = None) for f in tsca_cn}
wes_cn_dict = {f:pd.read_csv(f, sep="\t", index_col = None) for f in wes_cn}

# CN columns to display in tables
cn_col_names = ['external_id', 'Sample', 'condition','Chromosome', 'Start', 'End','Segment_Mean', 'Segment_Call', 'Num_Probes']

# Temporary directory for CN tables
cntempdir = '../temp/cclfreport/tables/'

# Local filepaths for the CN tables
local_cn_filepaths = [cntempdir + os.path.basename(i) for i in all_cn_tables]

In [ ]:
# Create single seg file containing all samples from targeted and WES

# Create dict of dicts: {participant_id: {external_id: seg file}}
seg_file_dict = dict()
seg_columns = ["external_id","Chromosome", "Start", "End", "Num_Probes", "Segment_Mean"]

# Read in tables and merge together into one giant seg file
seg_df = pd.concat((pd.read_csv(f,sep="\t", index_col = None) for f in all_cn_tables), sort=True)

# Convert to properly-formatted seg file and 
# convert the relative copy ratio to the log2(relative copy ratio)
seg_df = seg_df.loc[:,seg_columns]
seg_df["Segment_Mean"] = np.log2(seg_df["Segment_Mean"])

# Write table to a seg file (TSV format)
tmp_path = cntempdir + participant + ".tsv"
seg_df.to_csv(tmp_path, sep = "\t", index = False)

In [ ]:
# Convert to .cns format using CNVkit
converted = ! cnvkit.py import-seg {tmp_path} -d {cntempdir}

# Extract .cns filepaths
cns_files = []
# TODO: fix pattern so that has backslash before .cns. Otherwise, can end with cns instead of requiring .cns!
pattern = re.compile(r'{}.*.cns'.format(cntempdir))
for file in converted:
    cns_files += [re.search(pattern,file)[0]]
cns_files 

# Create list of targeted .cns filepaths
cns_targeted_ids = set(summary_df[summary_df.loc[:,"dataset"] == "targeted"].loc[:,"external_id"])
cns_targeted = [cntempdir+ext_id+".cns" for ext_id in cns_targeted_ids if ext_id in set(seg_df.loc[:,"external_id"])]
# cns_targeted = [path for path in cns_files if os.path.basename(path).split(".")[0] in cns_targeted_ids]

# Create list of WES .cns filepaths
cns_wes_ids = set(summary_df[summary_df.loc[:,"dataset"] == "WES"].loc[:,"external_id"])
cns_wes = [cntempdir+ext_id+".cns" for ext_id in cns_wes_ids if ext_id in set(seg_df.loc[:,"external_id"])]

## Copy number heat maps
There are two plots in this section, one for CN data from the targeted probe data and a second for CN data from WES data. To look at any one sample in more detail, you can look either at the corresponding horizontal CN plot in the next section titled "Copy number horizontal plots" or look at the CN table (see either the tables below or the TSV available at the link specified in the "Sample information and identifiers" section.

These tables are searchable and filterable.

In [ ]:
def make_cn_heat_map(cns_files, participant, disease, sample_source = "targeted", title = None, desaturated = False):
    segments = [cnvlib.read(f) for f in cns_files]
    plt.rcParams["font.size"] = 32
    plt.rcParams['figure.figsize'] = (32,max(6,len(segments)*3))

    if desaturated:
        ax = cnvlib.do_heatmap(segments, do_desaturate=True)
        if title is None:
            ax.set_title("Desaturated copy number heatmap for {} {} ({}) samples".format(sample_source, participant, disease))
        else:
            ax.set_title(title)
    
    else:
        ax = cnvlib.do_heatmap(segments, do_desaturate=False)
        if title is None:
            ax.set_title("Copy number heatmap for {} {} ({}) samples".format(sample_source, participant, disease))
        else:
            ax.set_title(title)

    plt.tight_layout()

### Targeted CN heat map

In [ ]:
# Create heat map from targeted .cns files
%matplotlib inline
make_cn_heat_map(cns_files = cns_targeted, participant = participant, disease = disease, sample_source = "targeted", desaturated = False)

### WES CN heat map

In [ ]:
# Create heat map from WES .cns files 
%matplotlib inline
make_cn_heat_map(cns_files = cns_wes, participant = participant, disease = disease, sample_source = "WES", title = "Copy number heatmap for PEDS204 WES samples", desaturated = False)

In [ ]:
# Create heat map from all .cns files (targeted and WES combined)
%matplotlib inline
make_cn_heat_map(cns_files = cns_files, participant = participant, disease = disease, sample_source = "all", title = None, desaturated = False)
make_cn_heat_map(cns_files = cns_files, participant = participant, disease = disease, sample_source = "all", title = None, desaturated = True)

## Copy number horizontal plots

Select the copy number plot you would like to display from the dropdown menu. The dropdown menu includes CN plots from both targeted probe (TSCA and TWIST) and WES data. The source of the data will be displayed on the title of the image. You can also refer to the table of all external IDs that maps each external ID to the source of the data (see "Sample information and identifiers").

The dropdown menu also includes a merged version of the horizontal copy number maps. This PNG file contains all the horizontal CN plots for the participant in a single place for ease of quick comparison.

In [ ]:
# horizontal_CN_plots = {os.path.basename(file)[:-4]:Image(file) for file in local_img_filepaths}
# image_names = ext_id_to_local_img.keys()
# display(horizontal_CN_plots[list(image_names).pop()])

In [ ]:
# Using Phil's code for dropdown menu in static HTML (display_card_view)
horizontal_plots = [Image(file) for file in local_img_filepaths]
display_card_view(labels = image_names, images = horizontal_plots)

In [ ]:
# # TODO: dropdown isn't exactly the external ID, but rather the name of the image file. See the creation of ext_id_to_local_img
# horizontal_CN_plots = {os.path.basename(file)[:-4]:Image(file) for file in local_img_filepaths}
# image_names = ext_id_to_local_img.keys()

# @interact
# def show_images(name=image_names):
#     gspath = ext_id_to_GS_img[name]
#     print("Image name: ", name)
#     print("File path:", gspath)
#     print("\nLink:", str(re.sub('gs://', 'https://console.cloud.google.com/storage/browser/', gspath)), "\n")
#     display(horizontal_CN_plots[name])

In [ ]:
# # TODO: dropdown isn't exactly the external ID, but rather the name of the image file. See the creation of ext_id_to_local_img
# horizontal_CN_plots = {os.path.basename(file)[:-4]:Image(file) for file in local_img_filepaths}
# image_names = ext_id_to_local_img.keys()

# # @interact
# def show_images(name=image_names):
#     gspath = ext_id_to_GS_img[name]
#     print("Image name: ", name)
#     print("File path:", gspath)
#     print("\nLink:", str(re.sub('gs://', 'https://console.cloud.google.com/storage/browser/', gspath)), "\n")
#     display(horizontal_CN_plots[name])

In [ ]:
# tab_contents = list(image_names)
# children = [widgets.Text(description=name) for name in tab_contents]
# children = [interactive(show_images, name=name) for name in tab_contents]
# tab = widgets.Accordion()
# tab.children = children
# # tab.titles = [str(i) for i in range(len(children))]
# for i in range(len(children)):
#     tab.set_title(i, tab_contents[i])
# tab

## Targeted CN table
The targeted copy number tables contains all the seg files for each sample that underwent targeted sequencing. The segment mean in the tables below represents (TODO: check) the log2(relative copy number). No pseudocount is included because none of the relative copy number values are ever exactly 0. (TODO: add)

In [ ]:
def display_table(file, file_table_dict, cols_to_use):
    # Print key information about the file
    print("Participant: ", str(os.path.basename(os.path.dirname(file))))
    print("\nFilepath: "+ file)
    print("\nLink:", str(re.sub('gs://', 'https://console.cloud.google.com/storage/browser/', file)))
    print("\nTo examine data for a particular external ID, filter the 'external_id' column.")
    
    # Get the TSV from the dict and display it
    df = file_table_dict[file]
    try:
        make_interactive_table(df, cols_to_include = cols_to_use)
    except:
        print("\nAt least one of this participant's mutation file is in a different format from the output of the newest pipeline. This data may be old, and have different column names. No filtering is performed on the displayed table, but you can add additional filters if desired:")
        make_interactive_table(df, cols_to_include = None)

In [ ]:
display_table(file = tsca_cn[0],file_table_dict = tsca_cn_dict, cols_to_use = cn_col_names)

## WES CN table

In [ ]:
display_table(file = wes_cn[0],file_table_dict = wes_cn_dict, cols_to_use = cn_col_names)

***
***

# Mutation data

In [ ]:
# Get the mutation TSVs
tsca_mut = ! gsutil ls -r {path}mutation.tsv
wes_mut = ! gsutil ls -r {path}**wes_mutations.tsv

# Create dictionary with filepaths as keys and pandas DF as the values
tsca_mut_dict = {f:pd.read_csv(f, sep="\t", index_col = None) for f in tsca_mut}
wes_mut_dict = {f:pd.read_csv(f, sep="\t", index_col = None) for f in wes_mut}

# Mutation TSV columns to display in tables
mut_col_names = ['external_id', 'Hugo_Symbol', 'Protein_Change','Variant_Classification', 'Variant_Type', 'tumor_f', 't_alt_count', 't_ref_count', 'COSMIC_total_alterations_in_gene',
                 'CGC_Tumor_Types_Somatic', 'CGC_Tumor_Types_Germline',
                 'Matched_Norm_Sample_Barcode','condition','Chromosome', 'Start_position', 'End_position','Genome_Change','keep']

Below are interactive tables containing *select* mutation information from the targeted probe data and the WES data. If there were multiple external IDs in either dataset, they have been combined into one table. The external_id column can be used to filter the data so only the mutations for a single external ID is displayed.

Note that this report only includes samples from the targeted data that pass the depth of coverage QC. Samples that did not pass this QC are not included in this report, and their data is not included in the Google bucket. A list of the samples that failed this QC is included earlier in this document (search for "Table: failed QC external IDs").

Also, note that the below tables have been filtered such that the keep column equals True. What this means is that only the variants that passed the filtering steps in the pipeline are included in the tables below. However, the raw mutation TSVs included in the Google bucket contain all the variants regardless of whether keep is True or False if you are interested in that information. This TSV will also contain columns explaining why a mutation was removed during filtration.

Generally speaking, if you are looking for more detailed information about why a mutation you expected to see was filtered out or if you want to get access to all of the columns available in the mutation TSV rather than the ones selected here, you can download the raw mutation TSV from the Google bucket.

## Targeted mutation table

In [ ]:
display_table(file = tsca_mut[0],file_table_dict = tsca_mut_dict, cols_to_use = mut_col_names)

## WES mutation table

In [ ]:
display_table(file = wes_mut[0],file_table_dict = wes_mut_dict, cols_to_use = mut_col_names)

In [ ]:
# TODO: Now we should delete all the data from the local temporary directories